In [ ]:
import os
import pandas as pd
from googletrans import Translator
from httpcore import TimeoutException

In [ ]:
def file_path(one_string):
    base_path = os.path.abspath(os.path.dirname("__file__"))
    archives = os.path.join(os.path.dirname(base_path), one_string)
    return archives

volcano_archives = os.path.join("datasets_input", "volcanics_input.csv")
volcano_spanish = os.path.join("datasets_output", "dataset_volcanic_pd.csv")

file_input = file_path(volcano_archives)
file_output = file_path(volcano_spanish)


In [ ]:
data_set_input = pd.read_csv(file_input, sep=';')


In [ ]:
def translator_volcano_type(data, retries=0):
    """Traduce el string pasado por parámetro al español"""
    translator = Translator()
    if retries < 3:
        try: 
           return translator.translate(data, dest='es').text.title()
        except TimeoutException:
            print("Tenemos una demora con la comunicación con Google, seguimos intentando") 
            retries += 1 
            translator_volcano_type(data, retries)
    
    else:
        raise Exception

In [ ]:
try:
    data_set_input = pd.read_csv(file_input, sep=';')

    # Convierto “Flag Tsunami” y “Flag Earthquake” a booleanos, considerando
    # que las celdas vacías corresponden al valor False
    new_flag_earthqueke = data_set_input['Flag Earthquake'].isna()
    new_flag_tsunami = data_set_input['Flag Tsunami'].isna()
    # traduzco el tipo de volcán al español
    volcano_spanish = data_set_input['Volcano Type']
    for k, v in volcano_spanish.items():
        volcano_spanish = volcano_spanish.replace([v],
                              translator_volcano_type(v))
    # Transformar la columna “Name” en una nueva columna “Name and country”
    # para que tenga una combinación del nombre del volcán con el país entre
    # paréntesis
    new_data = data_set_input['Volcano Name'].str.cat(data_set_input['Country'],
                   sep=' (') + ')'
    # genero una nueva variable para guardar los datos que se utilizarán
    # en las tarjetas
    data_set_input['Flag Tsunami'] = new_flag_tsunami
    data_set_input['Flag Earthquake'] = new_flag_earthqueke
    data_set_input['Volcano Name'] = new_data
    data_set_input['Volcano Type'] = volcano_spanish
    new_csv = data_set_input[['Year', 'Volcanic Explosivity Index',
                                  'Volcano Type', 'Flag Tsunami',
                                        'Flag Earthquake',
                                             'Volcano Name']]
    copy_new_csv = new_csv.copy()                                         
    copy_new_csv.rename(columns={"Volcano Name":"Name and Country"}, inplace=True)
    # escribo el archivo csv
    copy_new_csv.to_csv(file_output, index=False)
except FileNotFoundError:
    print('ERROR: No se ha encontrado el archivo.')
except Exception:
    print("Tuvimos problemas con la carga, necesita reiniciar el proceso")
else:
    print('Se ha creado el archivo correctamente.')